In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install google-cloud-vision
!pip install pdf2image
!apt-get install -y poppler-utils

In [ ]:
import os
import io
import csv
from pdf2image import convert_from_path
from google.cloud import vision

#configuración de Google Cloud Vision
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/Dataset/google-ocr.json"
client = vision.ImageAnnotatorClient()

# directorios de entrada (pdfs) y salida (OCR)
BASE_DIR = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish"
folders = {
    "esp": os.path.join(BASE_DIR, "esp"),
    "quz": os.path.join(BASE_DIR, "quz")
}
output_folder = os.path.join(BASE_DIR, "OCR_Texts")  # carpeta de salida
os.makedirs(output_folder, exist_ok=True)  # crear dicha carpeta si no existe

#función para extraer texto de una imagen con api de Google OCR
def extract_text_google_vision(image_path):
    """Extrae texto de una imagen usando Google Cloud Vision API"""
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if texts:
        return texts[0].description.strip()  # Eliminación de espacios extra

    return ""

# procesar pdfs en ambas carpetas
for lang, folder in folders.items():
    lang_output_folder = os.path.join(output_folder, lang)  # carpeta de salida por idioma
    os.makedirs(lang_output_folder, exist_ok=True)  # crear si no existe (por idioma)

    for file in sorted(os.listdir(folder)):
        if not file.endswith(".pdf"):
            continue

        pdf_path = os.path.join(folder, file)
        output_tsv_path = os.path.join(lang_output_folder, file.replace(".pdf", ".tsv"))

        # evitar reprocesar si ya existe
        if os.path.exists(output_tsv_path):
            print(f"Ya procesado: {output_tsv_path}")
            continue

        print(f"Procesando: {pdf_path} ...")

        # convertir PDF a imágenes para aplicar el OCR
        images = convert_from_path(pdf_path, dpi=300)

        #extraer texto de cada página
        extracted_data = []
        for i, image in enumerate(images):
            image_path = f"temp_page_{i+1}.jpg"
            image.save(image_path, "JPEG")  # guardar temporalmente en el espacio de trabajo de colab

            text = extract_text_google_vision(image_path)
            if text:
                extracted_data.append([i + 1, text])  # guardar número de página y texto correspondiente

            os.remove(image_path)  #eliminar imagen temporal

        #guardar en TSV solo si hay texto extraído
        if extracted_data:
            with open(output_tsv_path, "w", encoding="utf-8", newline="") as f:
                writer = csv.writer(f, delimiter="\t")
                writer.writerow(["pagina", "texto"])  #cabecera del tsv
                writer.writerows(extracted_data)

            print(f"Guardado: {output_tsv_path}")
        else:
            print(f"No se detectó texto en {pdf_path}.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import re

#función para limpiar texto:
#saltos de línea, normalizar espacios, eliminar símbolos no deseados (no alfabéticos)
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.replace('\n', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\w\s.,;:!?¿¡"()\'\-áéíóúñÁÉÍÓÚÑ]', '', text)
    return text

#definir rutas base para guardar versión limpia
base_input = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/OCR_Texts"
base_output = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/OCR_Cleaned_Texts"

for lang in ["esp", "quz"]:
    input_folder = os.path.join(base_input, lang)
    output_folder = os.path.join(base_output, lang)
    os.makedirs(output_folder, exist_ok=True)

    for filename in sorted(os.listdir(input_folder)):
        if not filename.endswith(".tsv"):
            continue

        file_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        try:
            df = pd.read_csv(file_path, sep="\t", encoding="utf-8")
            if "texto" not in df.columns:
                print(f"No se encontró columna 'texto' en: {filename}")
                continue

            df["texto"] = df["texto"].apply(clean_text)
            df.to_csv(output_path, sep="\t", index=False, encoding="utf-8")
            print(f"Archivo limpio guardado: {output_path}")

        except Exception as e:
            print(f"Error procesando {filename}: {e}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import re

#correcciones ortográficas para el quechua collao (glotalizado o aspirado) obtenidas de Manual de escritura
correcciones = {
    r'\bnuqa\b|\bnoqa\b': 'ñuqa',
    r'\bnuqawan\b|\bnoqawan\b': 'ñuqawan',
    r'\bqulqi\b|\bqolqe\b|\bqollqe\b': 'qullqi',
    r'\bsalqa\b|\bsalq’a\b': 'sallqa',
    r'\btanpa\b|\bt\'anpa\b': "t'ampa",
    r'\bmachkay\b|\bmashkay\b': 'maskhay',
    r'\bjawapi\b|\bqawapi\b': 'hawapi',
    r'\balqu\b|\balqo\b|\ballku\b': 'allqu',
    r'\bluqu\b|\bruqhu\b': 'ruqu',
    r'\bllihlla\b|\blliqlla\b': 'lliklla',
    r'\bqatun\b|\bjatun\b|\batun\b': 'hatun',
    r'\bjampi\b|\bqampi\b': 'hampi',
    r'\bwashka\b|\bwachka\b': 'waskha',
    r'\bkuchka\b|\bkushka\b': 'kuska',
    r'\bmijuna\b|\bmihuna\b': 'mikhuna',
    r'\blaphi\b|\brap’i\b|\blapi\b': 'raphi',
    r'\bhayay\b|\bjayay\b': 'qayay',
    r'\bwaq’ay\b|\bwahay\b|\bwajay\b': 'waqay',
    r'\bkilli\b|\bkhilli\b': 'qhilli',
    r'\bqhillu\b|\bk\'illu\b|\bkillu\b': "q'illu",
    r'\ballim\b': 'allin',
    r'\bwasimpi\b': 'wasinpi',
    r'\bpijchu\b|\bpihchu\b|\bpiqchu\b': 'pikchu',
    r'\bhuaita\b': 'wayta',
    r'\bmaitu\b': "mayt'u",
    r'\bhuauqei\b': 'wawqiy',
    r'\bccatimuhuay\b': 'qatimuway',
    r'\bhuaina\b': 'wayna',
    r'\byahuar\b': 'yawar',
    r'\bpishqa\b|\bpisqa\b|\bphisqa\b': 'pichqa',
    r'\bashka\b|\baskha\b': 'achka',
    r'\bch\'uqay\b|\bchhuqay\b': 'chuqay',
    r'\bwisq\'ay\b|\bwishqay\b': "wichq'ay",
    r'\bhuq\b|\bhuj\b': 'huk',
    r'\bkhayra\b|\bq\'ayra\b': "k'ayra",
    r'\bchiqaq\b': 'chiqap',
    r'\bllamphu\b': "llamp'u",
    r'\bhaykaq\b|\bhayk\'aq\b': "hayk'ap",
    r'\bhata\b|\bkhata\b': 'qhata',
    r'\busqaylla\b|\buthqaylla\b': 'utqaylla',
    r'\bqasqi\b': 'qatqi',
    r'\bt\'iqtiy\b|\btiqthiy\b|\btiqt\'iy\b': 'tiqtiy',
    r'\bllant\'u\b': "llanthu",
}

def corregir_texto(texto, log):
    palabras = texto.split()
    corregidas = []

    for palabra in palabras:
        original = palabra
        for patron, reemplazo in correcciones.items():
            if re.fullmatch(patron, palabra):
                log.append((original, reemplazo))
                palabra = reemplazo
                break
        corregidas.append(palabra)

    return ' '.join(corregidas)

#rutas de export
input_path = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/OCR_Cleaned_Texts/quz"
output_path = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/OCR_Standardized_Texts/quz"
log_path = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/OCR_Standardized_Texts/quz_logs"

os.makedirs(output_path, exist_ok=True)
os.makedirs(log_path, exist_ok=True)

#procesamiento de archivos
for archivo in sorted(os.listdir(input_path)):
    if not archivo.endswith(".tsv"):
        continue

    input_file = os.path.join(input_path, archivo)
    output_file = os.path.join(output_path, archivo)
    log_file = os.path.join(log_path, archivo.replace(".tsv", "_log.txt"))

    df = pd.read_csv(input_file, sep="\t", encoding="utf-8")
    if "texto" not in df.columns:
        print(f"No se encontró columna 'texto' en {archivo}")
        continue

    cambios = []
    df["texto"] = df["texto"].apply(lambda t: corregir_texto(t, cambios))
    df.to_csv(output_file, sep="\t", index=False, encoding="utf-8")

    with open(log_file, "w", encoding="utf-8") as f:
        for original, corregido in cambios:
            f.write(f"{original} → {corregido}\n")

    print(f"{archivo} corregido. Cambios: {len(cambios)}")

In [ ]:
import pandas as pd
import os
import re

#patrón de puntuación seguida de mayúscula
def split_oraciones(texto):
    return re.split(r'(?<=[.?!])\s+(?=[A-ZÁÉÍÓÚÑ])', texto.strip())

def calcular_metadata(tsv_path):
    try:
        df = pd.read_csv(tsv_path, sep="\t", encoding="utf-8")
        texto_completo = " ".join(df["texto"].dropna().tolist())
        oraciones = split_oraciones(texto_completo)
        palabras = texto_completo.split()
        palabras_unicas = set(palabras)

        return {
            "Documento": os.path.basename(tsv_path),
            "N° Oraciones": len(oraciones),
            "Palabras Totales": len(palabras),
            "Palabras Únicas": len(palabras_unicas)
        }
    except Exception as e:
        print(f"Error procesando {tsv_path}: {e}")
        return None

#carpeta de tsv (el input)
base_input = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/OCR_Cleaned_Texts"

resultados = []

for lang in ["esp", "quz"]:
    input_folder = os.path.join(base_input, lang)
    for filename in sorted(os.listdir(input_folder)):
        if not filename.endswith(".tsv"):
            continue
        path = os.path.join(input_folder, filename)
        metadata = calcular_metadata(path)
        if metadata:
            metadata["Idioma"] = lang
            resultados.append(metadata)

#guardar a csv
df_resultado = pd.DataFrame(resultados)
output_csv = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/outputs/metadata_r1.csv"
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df_resultado.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Metadata guardada en: {output_csv}")

In [ ]:
import pandas as pd

#leer la metadata ya guardada
metadata_path = "/content/drive/MyDrive/Dataset/JW_Org_Quechua_Spanish/outputs/metadata_r1.csv"
df = pd.read_csv(metadata_path)
#Resumen general
resumen = {
    "Documentos procesados": len(df),
    "Idiomas": df["Idioma"].unique().tolist(),
    "Oraciones (promedio)": round(df["N° Oraciones"].mean(), 2),
    "Oraciones (mínimo)": df["N° Oraciones"].min(),
    "Oraciones (máximo)": df["N° Oraciones"].max(),
    "Palabras Totales (promedio)": round(df["Palabras Totales"].mean(), 2),
    "Palabras Totales (mínimo)": df["Palabras Totales"].min(),
    "Palabras Totales (máximo)": df["Palabras Totales"].max(),
    "Palabras Únicas (promedio)": round(df["Palabras Únicas"].mean(), 2),
    "Palabras Únicas (mínimo)": df["Palabras Únicas"].min(),
    "Palabras Únicas (máximo)": df["Palabras Únicas"].max()
}
pd.DataFrame([resumen])

In [ ]:
#Resumen específico por idioma
resumen_por_idioma = df.groupby("Idioma").agg({
    "Documento": "count",
    "N° Oraciones": ["mean", "min", "max"],
    "Palabras Totales": ["mean", "min", "max"],
    "Palabras Únicas": ["mean", "min", "max"]
}).reset_index()

resumen_por_idioma.columns = [
    "Idioma", "Documentos",
    "Oraciones (prom)", "Oraciones (min)", "Oraciones (max)",
    "Palabras Totales (prom)", "Palabras Totales (min)", "Palabras Totales (max)",
    "Palabras Únicas (prom)", "Palabras Únicas (min)", "Palabras Únicas (max)"
]
resumen_por_idioma = resumen_por_idioma.round(2)
resumen_por_idioma